In [ ]:
import os
from pathlib import Path 
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, regularizers

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/dogs_and_cats

Mounted at /gdrive
/gdrive/MyDrive/dogs_and_cats


In [ ]:
import zipfile
with zipfile.ZipFile("/gdrive/MyDrive/dogs_and_cats/dogs-vs-cats.zip","r") as zip_ref:
    zip_ref.extractall("/gdrive/MyDrive/dogs_and_cats")
with zipfile.ZipFile("/gdrive/MyDrive/dogs_and_cats/train.zip","r") as zip_ref:
    zip_ref.extractall("/gdrive/MyDrive/dogs_and_cats")
with zipfile.ZipFile("/gdrive/MyDrive/dogs_and_cats/test1.zip","r") as zip_ref:
    zip_ref.extractall("/gdrive/MyDrive/dogs_and_cats")

In [ ]:
DATA_DIR = Path("/gdrive/MyDrive/dogs_and_cats/")
IMAGE_DIR =  DATA_DIR / "train/"
MODEL_PATH = DATA_DIR / "savel_model/model1/"
RANDOM_SEED = 42
BATCH_SIZE = 64 
VALID_SIZE = 0.2
IMAGE_SIZE = (200, 200)
N_CHANNELS = 3
N_CLASSES = 2
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 0.001
N_EPOCHS = 100

In [ ]:
train_labels = list()
for name_image in os.listdir(IMAGE_DIR):
  name = name_image.split(".")[0]
  if name == "dog" :
    train_labels.append(1)
  else : train_labels.append(0)

In [ ]:
df = pd.DataFrame({"image_ids":os.listdir(IMAGE_DIR), "labels": train_labels})
df.labels = df.labels.replace({0:"cat", 1:"dog"})
train_df, valid_df = train_test_split(
        df, test_size=VALID_SIZE, random_state=RANDOM_SEED, stratify=df.labels,
)
train_df.reset_index(drop=True ,inplace=True)
valid_df.reset_index(drop=True ,inplace=True)


In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_dataset = train_datagen.flow_from_dataframe(
    train_df, 
    IMAGE_DIR, 
    x_col='image_ids',
    y_col='labels',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

Found 20000 validated image filenames belonging to 2 classes.


In [ ]:
valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
valid_dataset = valid_datagen.flow_from_dataframe(
    valid_df, 
    IMAGE_DIR, 
    x_col='image_ids',
    y_col='labels',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE
)

Found 5000 validated image filenames belonging to 2 classes.


In [ ]:
model = keras.Sequential([
    keras.Input(shape=(*IMAGE_SIZE, N_CHANNELS)),
    layers.Conv2D(32, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(),
    layers.Dropout(0.25),

    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(),
    layers.Dropout(0.25),

    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation = "relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(2, activation = "softmax"),

])


In [ ]:
model.load_weights(MODEL_PATH)

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 200, 200, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 100, 100, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 100, 100, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 100, 100, 64)     2

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(MODEL_PATH, save_weights_only=True, verbose = 1),
    keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=1),
]

In [ ]:
model.fit(train_dataset, validation_data=valid_dataset, callbacks=callbacks, epochs=N_EPOCHS)

Epoch 1/100
 86/313 [=======>......................] - ETA: 2:35:28 - loss: 0.3175 - accuracy: 0.8614